In [3]:
#!unzip /content/train.zip

Archive:  /content/train.zip
   creating: modified_brown/
  inflating: modified_brown/ca01     
  inflating: modified_brown/ca02     
  inflating: modified_brown/ca03     
  inflating: modified_brown/ca04     
  inflating: modified_brown/ca05     
  inflating: modified_brown/ca06     
  inflating: modified_brown/ca07     
  inflating: modified_brown/ca08     
  inflating: modified_brown/ca09     
  inflating: modified_brown/ca10     
  inflating: modified_brown/ca11     
  inflating: modified_brown/ca12     
  inflating: modified_brown/ca13     
  inflating: modified_brown/ca14     
  inflating: modified_brown/ca15     
  inflating: modified_brown/ca16     
  inflating: modified_brown/ca17     
  inflating: modified_brown/ca18     
  inflating: modified_brown/ca19     
  inflating: modified_brown/ca20     
  inflating: modified_brown/ca21     
  inflating: modified_brown/ca22     
  inflating: modified_brown/ca23     
  inflating: modified_brown/ca24     
  inflating: modified_brown/ca

In [4]:
import os
import sys
import numpy as np

In [6]:
def load_corpus(path):
    sentences = []
    count = 0
    for sample in os.listdir(path):
      count += 1
      print(count)
      with open(path+"/" +sample, "r", errors="ignore") as f:
        data = f.read()
      for line in data:
        sentence = data.strip().split(" ")
        tagged_sentence = []
        for word in sentence:
          tagged_sentence.append(tuple(word.split("/")))
      sentences.append(tagged_sentence)
    return sentences

#corpus = load_corpus("/content/modified_brown")

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [11]:
class HMMTagger:

    def __init__(self):
        self.tag_probabilities = {}
        self.transition_probabilities = {}
        self.emission_probabilities = {}
        self.tags = {"ADJECTIVE": 1, "NOUN": 2, "PUNCT":3, "PRONOUN":4, "VERB":5, "PREPOSITION":6, "DETERMINER":7, "CONJUNCTION":8, "ADVERB":9, "NUMBER":10, "X":11}
        self.num_tags = {}


    def initialize_probabilities(self, sentences):
        total_number_of_sentences = len(sentences)
        
        #number of tags
        num_tags = {}
        for sentence in sentences:
          for entry in sentence:
            tag = entry[1]
            if tag in num_tags:
              num_tags[tag] += 1
            else:
              num_tags[tag] = 1

        #tag probabilities
        tag_probabilities = {}
        for sentence in sentences:
          tag = sentence[0][1]
          if tag in tag_probabilities:
            tag_probabilities[tag] += 1
          else:
            tag_probabilities[tag] = 1
        for tag in tag_probabilities.keys():
          tag_probabilities[tag] = tag_probabilities[tag]/total_number_of_sentences
        
        #transition probabilities
        transition_probabilities = {}
        for sentence in sentences:
          for i in range(1, len(sentence)):
            transition = (sentence[i-1][1], sentence[i][1])
            if transition in transition_probabilities:
              transition_probabilities[transition] += 1
            else:
              transition_probabilities[transition] = 1
        for transition in transition_probabilities.keys():
          transition_probabilities[transition] = transition_probabilities[transition]/num_tags[transition[0]]

        #emission probabilities
        emission_probabilities = {}
        for sentence in sentences:
          for entry in sentence:
            entry = (entry[0].lower(), entry[1])
            if entry in emission_probabilities:
              emission_probabilities[entry] += 1
            else:
              emission_probabilities[entry] = 1
        for entry in emission_probabilities.keys():
          emission_probabilities[entry] = emission_probabilities[entry] / num_tags[entry[1]]

        self.num_tags = num_tags
        self.tag_probabilities = tag_probabilities
        self.transition_probabilities =transition_probabilities
        self.emission_probabilities = emission_probabilities 
    
    def viterbi_decode(self, sentence):
        #print(self.num_tags)
        #print(self.tag_probabilities)
        #print(self.transition_probabilities)
        #print(self.emission_probabilities)
        sentence = sentence.strip().split(" ")
        most_probable_tags = []
        trails = {}
        trails[0] = {}
        for tag in self.tags.keys():
          emission_probability =  self.emission_probabilities[(sentence[0].lower(), tag)] if (sentence[0].lower(), tag) in self.emission_probabilities else 1e-10
          tag_probability = self.tag_probabilities[tag] if tag in self.tag_probabilities else 1e-10
          if emission_probability*tag_probability != 0:
            trails[0][tag]= [tag_probability * emission_probability, "START" ]
        for i in range(1, len(sentence)):
          token = sentence[i].lower()
          trails[i] = {}
          for tag in self.tags.keys():
            trails[i][tag] = [0, ""]
            for prev_tag in trails[i-1].keys():
              emission_probability =  self.emission_probabilities[(sentence[i].lower(), tag)] if (sentence[i].lower(), tag) in self.emission_probabilities else 1e-10
              transition_probability = self.transition_probabilities[(prev_tag, tag)] if (prev_tag, tag) in self.transition_probabilities else 1e-10
              prev_probability = trails[i-1][prev_tag][0]
              if emission_probability*transition_probability*prev_probability > trails[i][tag][0]:
                trails[i][tag] = [emission_probability*transition_probability*prev_probability, prev_tag]
            if trails[i][tag][0] == 0:
              trails[i].pop(tag)
        
        prediction = []
        num_trails = len(sentence)
        max_probability = 0
        predicted_tag = ""
        prev_tag = ""
        for tag in trails[num_trails - 1].keys():
          if trails[num_trails - 1][tag][0] > max_probability:
            max_probability = trails[num_trails - 1][tag][0]
            predicted_tag = tag
            prev_tag = trails[num_trails - 1][tag][1]
        prediction.insert(0, predicted_tag)
        
        #print(trails)
        predicted_tag = prev_tag
        prediction.insert(0, predicted_tag)
        for i in range(len(sentence)-2, 0, -1):
          #print(prediction)
          prediction.insert(0, trails[i][predicted_tag][1])
          predicted_tag = trails[i][predicted_tag][1]
        return prediction

# Initialize the tagger class
tagger = HMMTagger()

# Read a corpus and learn from it
folder_name = input("Input path: ")
sentences = load_corpus(folder_name)
#sentences = corpus
tagger.initialize_probabilities(sentences)

# Test
sentence = "the planet jupiter and its moons are in effect a mini solar system ."
result = tagger.viterbi_decode(sentence)
print(sentence)
print(result)

sentence = "computers process programs accurately ."
result = tagger.viterbi_decode(sentence)
print(sentence)
print(result)



the planet jupiter and its moons are in effect a mini solar system .
['DETERMINER', 'NOUN', 'NOUN', 'CONJUNCTION', 'PRONOUN', 'NOUN', 'VERB', 'PREPOSITION', 'NOUN', 'DETERMINER', 'ADJECTIVE', 'ADJECTIVE', 'NOUN', 'PUNCT']
computers process programs accurately .
['NOUN', 'NOUN', 'NOUN', 'ADVERB', 'PUNCT']
